In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import pandas as pd
from os.path import abspath

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv(abspath('../../data/MSFT/MSFT-5-Year.csv'), index_col=0, parse_dates=['Date'])

df.columns.name = df.index.name
df.index.name = None

df = df.filter(['Date', 'Open'])
print(df.shape)
df.head()

In [ ]:
# Split the df into a training set and a test set
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_set, test_set = train_test_split(df, test_size=0.05, shuffle=False)

n_features = 1

In [ ]:
# Plot the training and testing data
train_set.Open.plot(label='Training Data')
test_set.Open.plot(label='Test Data')
plt.legend()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler()
train_scaled = transformer.fit_transform(train_set)
test_scaled = transformer.fit_transform(test_set)

In [ ]:
import tensorflow as tf

#List attached GPU devices
print(tf.config.list_physical_devices('GPU'))

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # type: ignore

n_input = 60

train_generator=TimeseriesGenerator(train_scaled,
                                     train_scaled,
                                      length=n_input,
                                      batch_size=32)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

model = Sequential()
        
#Add first LSTM layer
model.add(LSTM(units=20, input_shape=(n_input, n_features)))
model.add(Dense(units=1))
model.summary()

In [ ]:
# Create an EarlyStopping callback
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10)

In [ ]:
from CustomAdam import CustomAdam

model.compile(optimizer=CustomAdam(), loss='mean_squared_error')
model.fit(train_generator, epochs=25, callbacks=[early_stopping])

In [ ]:
model.save(abspath('../../logs/MSFT-5-Year/'))

In [ ]:
# # method to check if a file exists
# def file_exists(file_path):
#     import os
#     return os.path.isfile(file_path)

# from keras.models import load_model

# if (file_exists('/Users/aditya/Programming/stock-market-predictions/stock_market_predictions/forecasting/LSTM/MSFT-5-Year-LSTM.h5')):
#     model = load_model('/Users/aditya/Programming/stock-market-predictions/stock_market_predictions/forecasting/LSTM/MSFT-5-Year-LSTM.h5')
# else:
#     print("File does not exist")

In [ ]:
inputs = df[len(df)-len(test_set)-60 : ].values
inputs = inputs.reshape(-1,1)
inputs = transformer.transform(inputs)

In [ ]:
import numpy as np

x_test = []
for i in range(60,len(inputs)):
    x_test.append(inputs[i-60:i, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
print(x_test.shape)

In [ ]:
predicted = model.predict(x_test)
predicted = transformer.inverse_transform(predicted)

In [ ]:
print(predicted.shape)

# Convert the predicted values to a dataframe
predicted_df = pd.DataFrame(predicted)

# Rename the column to "Open"
predicted_df.columns = ['Open']

# Add a new column called "Date" that has the same index as the other dataframe
predicted_df['Date'] = df[len(df)-len(test_set)-0 : ].index

# Set the 'Date' column as the index
predicted_df = predicted_df.set_index('Date')

predicted_df.head()

In [ ]:
# Plot the predicted values
predicted_df.plot(label='Predicted')

In [ ]:
def plot_predictions(test, predicted):
    plt.figure(4, figsize=(10,6))
    plt.plot(test, color="red", label="real MSFT stock price")
    plt.plot(predicted, color="blue", label="predicted stock price")
    plt.title("MSFT stock price prediction")
    plt.xlabel("time")
    plt.ylabel("MSFT stock price")
    plt.legend()
    plt.show()

plot_predictions(test_set, predicted_df)

In [ ]:
import math
from sklearn.metrics import mean_squared_error

def return_rmse(test, predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("the root mean squared error is: {}".format(rmse))

return_rmse(test_set, predicted)